In [1]:
import paramiko
import datetime
import pandas as pd
import numpy as np
import os

print(os.getcwd())
print(datetime.datetime.now())

/home/jian/Projects/Big_Lots/Predictive_Model/LR_Mapping_File
2020-05-28 12:17:41.029728


In [3]:
host_66 = "66.55.157.58" #hard-coded
port_66 = 22
transport_66 = paramiko.Transport((host_66, port_66))

password_66 = "jubaplus2019" #hard-coded
username_66 = "jian" #hard-coded
transport_66.connect(username = username_66, password = password_66)
sftp_66 = paramiko.SFTPClient.from_transport(transport_66)

In [4]:
folder_remote_66_others="/data/jian/BL_LR_download/others/"
list_mappings=sftp_66.listdir(folder_remote_66_others)
list_mappings=[x for x in list_mappings if "mapping" in x]
df_remote_mapping=pd.DataFrame({"remote_base_name":list_mappings},index=[0]*len(list_mappings))
df_remote_mapping['Quarter']=df_remote_mapping['remote_base_name'].apply(lambda x: x.split("_mapping_file_")[1][:6])
df_remote_mapping['remote_mt_time']=df_remote_mapping['remote_base_name'].apply(lambda x: sftp_66.stat(folder_remote_66_others+x).st_mtime)
df_remote_mapping=df_remote_mapping.sort_values(["remote_base_name","remote_mt_time"],ascending=[False,False])
df_remote_mapping=df_remote_mapping.drop_duplicates("Quarter")
df_remote_mapping

,remote_base_name,Quarter,remote_mt_time
0,CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-1...,2020Q1,1589630340


In [5]:
quarters_already_copied=os.listdir("./")
quarters_already_copied=[x for x in quarters_already_copied if "_mapping_file_" in x]
quarters_already_copied=[x.split("_mapping_file_")[1][:6] for x in quarters_already_copied]
quarters_already_copied=set(quarters_already_copied)
quarters_already_copied

set()

In [6]:
df_remote_mapping=df_remote_mapping[~df_remote_mapping['Quarter'].isin(quarters_already_copied)]
df_remote_mapping

,remote_base_name,Quarter,remote_mt_time
0,CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-1...,2020Q1,1589630340


In [7]:
df_remote_mapping['remote_base_name'][0]

'CL_BigLots_BL_mapping_file_2020Q1_JL_2020-05-13_20200514_172449_0000.psv.gz'

In [8]:
df_remote_mapping=df_remote_mapping.drop_duplicates("Quarter")